In [48]:
import os, glob
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font

# Carpeta base donde están los archivos
BASE_DIR = r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson"
os.makedirs(BASE_DIR, exist_ok=True)

print("Directorio base:", BASE_DIR)



Directorio base: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson


In [44]:
#Función para  leer, limpiar, guardar e integrar archivos de todos los gestores de un tipo especifico.

def appendfiles(filenamepattern, base_dir=BASE_DIR):
    searchpattern = os.path.join(base_dir, f"{filenamepattern}_*.txt")
    files = glob.glob(searchpattern)
    print("Buscando:", searchpattern)
    print("Archivos encontrados:", files)
    if not files:
        print("⚠️ No se encontraron archivos.")
        return None
    out_name = f"Integrated_{filenamepattern}_files.txt"
    out_path = os.path.join(base_dir, out_name)
    with open(out_path, "w", encoding="utf-8") as out:
        for name in files:
            with open(name, "r", encoding="utf-8") as f:
                out.write(f.read())
                if not f.read().endswith("\n"):
                    out.write("\n")
            print("Agregado:", name)
    return out_name


def cleanfile(filename, base_dir=BASE_DIR):
    if filename is None:
        return None
    in_path = os.path.join(base_dir, filename)
    out_name = f"Clean_{filename}"
    out_path = os.path.join(base_dir, out_name)
    ignore = ("SubNetwork,", "instance(s)", "NodeId")
    with open(in_path, "r", encoding="utf-8") as fin, open(out_path, "w", encoding="utf-8") as fout:
        for line in fin:
            if any(p in line for p in ignore):
                continue
            fout.write(line)
    return out_name


def convert_to_excel(cleanfile_name, base_dir=BASE_DIR):
    if cleanfile_name is None:
        return None
    tsv_path = os.path.join(base_dir, cleanfile_name)
    df = pd.read_csv(tsv_path, delimiter="\t", header=None)
    out_xlsx = os.path.join(base_dir, f"Converted_{cleanfile_name}.xlsx")
    df.to_excel(out_xlsx, index=False, header=None)
    return out_xlsx


In [45]:
# 2.1 Procesar EUtranCellFDD
fn_cells = appendfiles("EUtranCellFDD")
clean_cells = cleanfile(fn_cells)
xlsx_cells = convert_to_excel(clean_cells)

# 2.2 Procesar ENodeBFunction
fn_nodeb = appendfiles("ENodeBFunction")
clean_nodeb = cleanfile(fn_nodeb)
xlsx_nodeb = convert_to_excel(clean_nodeb)

# 2.3 Cargar ambos DataFrames
df_cells = pd.read_excel(xlsx_cells, header=None, dtype=str)   # todo como texto
df_nodeb = pd.read_excel(xlsx_nodeb, header=None, dtype=str)
df_nodeb.columns = ["NodeId", "ENodeBFunctionId", "eNBIdnew"]

# 2.4 Reordenar columnas (mover columna B al final)
cols = df_cells.columns.tolist()
if len(cols) >= 2:
    colB = cols.pop(1)   # saca B
    cols.append(colB)    # pone al final
    df_cells = df_cells[cols]

# 2.5 Definir headers
headers = [
    "eNodeB Name", "CellName", "activePlmnList_mcc", "additionalPlmnList_mcc",
    "administrativeState", "cellBarred", "cellId", "cellSubscriptionCapacity",
    "channelSelectionSetSize", "dlChannelBandwidth", "earfcndl", "earfcnul",
    "freqBand", "noOfPucchCqiUsers", "noOfPucchSrUsers", "operationalState",
    "physicalLayerCellIdGroup", "physicalLayerSubCellId", "sectorCarrierRef",
    "tac", "timeOfLastModification", "ulChannelBandwidth", "eNBId",
    "eNodeB Name Unique", "LAT", "LON", "PCI", "AT&T_Site_Name",
    "MOCN Activo por Celda", "Al menos una celda de MOCN encendida", "MME TEF"
]

# Ajustar si no coincide el número de columnas
n_cols = df_cells.shape[1]
target = len(headers)

if n_cols < target:
    # Agrega columnas vacías para que no se pierda ningún header (incl. "MME TEF")
    for i in range(n_cols, target):
        df_cells[i] = pd.NA
    # Reindexa por índice numérico para asegurar el orden 0..target-1
    df_cells = df_cells.reindex(columns=range(target))
elif n_cols > target:
    # Si trae más columnas de la cuenta, conserva solo las primeras 'target'
    df_cells = df_cells.iloc[:, :target]

# Ahora sí, asigna los nombres de columna
df_cells.columns = headers
# 2.6 Hacer el join (map)
enbid_map = df_nodeb.set_index("NodeId")["eNBIdnew"]
df_cells["eNBId"] = df_cells["eNodeB Name"].map(enbid_map)

# 2.7 Guardar versión raw sin encabezados
final_raw = os.path.join(BASE_DIR, "Datos_Modified_raw.xlsx")
df_cells.to_excel(final_raw, index=False, header=False)
print("Archivo raw guardado:", final_raw)


Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_*.txt
Archivos encontrados: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\EUtranCellFDD_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\EUtranCellFDD_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_9.txt


C:\Users\SCaracoza\AppData\Local\Temp\ipykernel_15460\1849020420.py:42: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(tsv_path, delimiter="\t", header=None)


Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_*.txt
Archivos encontrados: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\ENodeBFunction_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\ENodeBFunction_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_9.txt
Archivo raw guardado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Datos_Modified_raw.xlsx


In [47]:
def apply_final_headers(xlsx_in, xlsx_out, headers):
    wb = load_workbook(xlsx_in)
    ws = wb.active

    # Insertar fila 1
    ws.insert_rows(1)

    # Escribir headers con formato vertical
    for col_idx, header in enumerate(headers, start=1):
        c = ws.cell(row=1, column=col_idx, value=header)
        c.font = Font(name="Aptos Narrow", size=11, bold=False)  # sin negritas
        c.alignment = Alignment(textRotation=90, horizontal="center", vertical="bottom", wrap_text=True)

    ws.freeze_panes = "A2"  # opcional: congela encabezados
    wb.save(xlsx_out)

final_pretty = os.path.join(BASE_DIR, "Datos_Modified.xlsx")
apply_final_headers(final_raw, final_pretty, headers)

print("✅ Archivo final listo:", final_pretty)


✅ Archivo final listo: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Datos_Modified.xlsx
